<h1><center>Machine Learning: Clustering</center></h1>

## What is Clustering?
- Clustering is a general problem that we need to divide the whole dataset into several small groups (clusters) so that the points within each group are quite similar in certain ways. There are lots of daily examples around us:

    1. Market Segmentation: Divide a broad business market into small segments so within each segments the most profitable characteristic could be discovered. After knowing this, they could select some of segments to become target markets.
    2. Recommendation System: The system could collect your search history and then recommend similar products/videos/articles to you on the main page.
    3. Image Segmentation: This is the process when dividing the whole all pixels into different non-overlapping sub groups and the union of them is the whole image.

- By studying the clustering algorithm, we will also understand more about the unsupervised learning, a specific type of algorithm in the machine learining.

## What is Unsupervised Learning?
Unsupervised Learning is a specific type of machine learning algorithm, which tends to figure out the deep and hidden patterns from unlabeled dataset. 
